Library


In [45]:
import math
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import scipy.fftpack
import librosa
import soundfile as sf
from scipy.signal import resample_poly
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.tree import DecisionTreeClassifier, export_text

Main Path


In [46]:
MAIN_PATH = r"C:\Users\Lulay\Documents\GitHub\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"
# MAIN_PATH = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"

Konstanta


In [47]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)
RANDOM_STATE = 21
BUFFER_SIZE = 1000

Random Seed


In [48]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Prapemrosesan


In [49]:
def load_file_audio(path):
    audio, sr = sf.read(path)
    return np.array(audio), sr

def prapemrosesan_downmixing(audio):
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    return audio.astype(np.float32)

def prapemrosesan_resampling(audio, sr):
    if sr == SAMPLE_RATE:
        return audio.copy(), SAMPLE_RATE
    
    ratio = SAMPLE_RATE / sr
    n_samples = int(np.round(len(audio) * ratio))
    
    x_old = np.linspace(0, 1, len(audio))
    x_new = np.linspace(0, 1, n_samples)
    return np.interp(x_new, x_old, audio), SAMPLE_RATE

def prapemrosesan_padding(audio):
    if np.mod(audio.shape[0], SEGMENT_DURATION) != 0:
        padding = SEGMENT_DURATION - (audio.shape[0] % SEGMENT_DURATION)
        audio = np.pad(audio, (0, padding))
    return audio

def prapemrosesan_splitting(audio):
    num_segments = int(np.floor((len(audio) - SEGMENT_DURATION) / OVERLAP_DURATION)) + 1
    segments = []

    for i in range(num_segments):
        start = int(i * OVERLAP_DURATION)
        end = int(start + SEGMENT_DURATION)
        segment = audio[start:end]
        if len(segment) < SEGMENT_DURATION:
            segment = np.pad(segment, (0, SEGMENT_DURATION - len(segment)), mode='constant')
        segments.append(segment)

    return np.array(segments)

# Ekstraksi Fitur


In [50]:
def get_rms(segment):
    return np.sqrt(np.mean(segment ** 2))

def get_zcr(segment):
    return np.sum(np.abs(np.diff(np.signbit(segment)))) / (len(segment) / SAMPLE_RATE)

def get_lms(segment):
    mel_spec = librosa.feature.melspectrogram(y=segment, sr=SAMPLE_RATE)
    return np.mean(mel_spec)

# Dataset


Raw


In [51]:
with open(f"{MAIN_PATH}/dataset_preprocessed.pkl", "rb") as f:
    dataset = pickle.load(f)

In [52]:
len(dataset)

793

Normalisasi, Split, dan Batching


In [53]:
dataset[0][1:]

[[0.0, 1.0],
 2,
 0.012409585523878887,
 11094.0,
 array([[2.16197470e-13, 1.03119000e-09, 1.56593188e-07, ...,
         7.38014642e-07, 4.86190822e-07, 3.69650615e-05],
        [1.71594322e-13, 1.73647630e-09, 6.84359839e-08, ...,
         1.20641453e-06, 1.28787660e-06, 3.60378650e-05],
        [2.33699181e-13, 1.29767993e-09, 1.06911272e-08, ...,
         1.49087633e-06, 7.68526166e-06, 5.72646908e-05],
        ...,
        [3.98201299e-13, 4.03694204e-11, 6.71124721e-10, ...,
         2.79048998e-07, 2.46822598e-07, 8.22193067e-07],
        [5.72516160e-13, 3.12001303e-11, 4.63515168e-10, ...,
         3.61625092e-07, 2.90140389e-07, 8.36865461e-07],
        [8.76847812e-13, 2.16880748e-11, 3.50852229e-10, ...,
         2.80869441e-07, 2.98575333e-07, 8.50766193e-07]])]

In [59]:
X = [[data[3], data[4]] for data in dataset if data[2] != 2]
y = [int(data[2]) for data in dataset if data[2] != 2]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE)

In [62]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [63]:
display(
    X_train_scaled.shape,
    X_test_scaled.shape,
    X_val_scaled.shape
)

(265, 2)

(33, 2)

(34, 2)

# LGBM


Inisialisasi


In [64]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)

training


In [65]:
lgbm_model.fit(X_train_scaled, y_train)

LGBMClassifier(random_state=21, verbose=-1)

# Decision Tree


Inisialisasi


In [104]:
dt_model = DecisionTreeClassifier(random_state=RANDOM_STATE)

training


In [105]:
dt_model.fit(X_train_input, y_train)

DecisionTreeClassifier(random_state=21)

# CNN


Inisialisasi


In [106]:
# class CNN(nn.Module):
#     def __init__(self, num_classes=10):
#         super(CNN, self).__init__()
#         test_input = np.random.rand(SAMPLE_RATE * SEGMENT_DURATION)
#         lms = get_lms(test_input)
#         in_channels = 1
#         self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(128)
#         self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(256)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.gap = nn.AdaptiveAvgPool2d(1)
#         self.fc1 = nn.Linear(256, 256)
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, num_classes)
#     def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))
#         x = self.pool(F.relu(self.bn3(self.conv3(x))))
#         x = self.gap(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

training


# Conformer


Inisialisasi


In [107]:
# config = ConformerConfig(
#     hidden_size=144,
#     num_attention_heads=4,
#     intermediate_size=576,
#     conv_kernel_size=32,
#     num_hidden_layers=8,
#     input_feat_per_channel=n_mels,
#     input_channels=1,
#     max_position_embeddings=time_steps
# )

# conformer_model = TFConformerModel(config)

# input_layer = tf.keras.layers.Input(shape=(n_mels, time_steps))
# expand_dim = tf.keras.layers.Reshape((1, n_mels, time_steps))(input_layer)
# conv_proj = tf.keras.layers.Conv2D(
#     filters=config.hidden_size,
#     kernel_size=(3, 3),
#     padding='same',
#     activation='relu'
# )(expand_dim)
# squeeze_dim = tf.keras.layers.Reshape((n_mels, time_steps, config.hidden_size))(conv_proj)
# conformer_output = conformer_model(squeeze_dim).last_hidden_state
# gap = tf.keras.layers.GlobalAveragePooling1D()(conformer_output)
# output = tf.keras.layers.Dense(num_classes, activation='softmax')(gap)

# conformer_model = tf.keras.Model(inputs=input_layer, outputs=output)

training


# Model Lain


inisialisasi


In [108]:
models = {
    "rf": RandomForestClassifier(random_state=RANDOM_STATE),
    "xg": XGBClassifier(random_state=RANDOM_STATE),
    "cat": CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
    "gb": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "lr": LogisticRegression(random_state=RANDOM_STATE),
    "knn": KNeighborsClassifier(),
    "dt": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "svm": SVC(random_state=RANDOM_STATE)
}

training


In [109]:
results = {}

for model_name, model in models.items():
    model.fit(X_train_input, y_train)
    y_pred = model.predict(X_test_input)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    results[model_name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm.tolist()
    }

# Evaluasi


LGBM


In [70]:
y_pred = lgbm_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = lgbm_model.predict(X_test_scaled)

print("\n")
print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Validasi
Accuracy: 0.8823529411764706
Precision: 0.7777777777777778
Recall: 0.7777777777777778
F1 Score: 0.7777777777777778
Confusion Matrix:
 [[23  2]
 [ 2  7]]


Test
Accuracy: 0.9393939393939394
Precision: 0.9285714285714286
Recall: 0.9285714285714286
F1 Score: 0.9285714285714286
Confusion Matrix:
 [[18  1]
 [ 1 13]]


CNN


Conformer


Descision Tree


In [111]:
y_pred = dt_model.predict(X_val_input)

print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = dt_model.predict(X_test_input)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Accuracy: 0.9454545454545454
Precision: 0.9523809523809523
Recall: 0.9090909090909091
F1 Score: 0.9302325581395349
Confusion Matrix:
 [[32  1]
 [ 2 20]]
Accuracy: 0.8727272727272727
Precision: 0.8888888888888888
Recall: 0.7619047619047619
F1 Score: 0.8205128205128205
Confusion Matrix:
 [[32  2]
 [ 5 16]]


In [112]:
feature_names = ['RMS', 'ZCR']
tree_rules = export_text(dt_model, feature_names=feature_names)
print("Extracted Decision Rules:")
print(tree_rules)

Extracted Decision Rules:
|--- ZCR <= -0.57
|   |--- RMS <= -0.22
|   |   |--- RMS <= -0.66
|   |   |   |--- class: 1
|   |   |--- RMS >  -0.66
|   |   |   |--- RMS <= -0.65
|   |   |   |   |--- class: 0
|   |   |   |--- RMS >  -0.65
|   |   |   |   |--- class: 1
|   |--- RMS >  -0.22
|   |   |--- ZCR <= -0.80
|   |   |   |--- ZCR <= -1.16
|   |   |   |   |--- ZCR <= -1.21
|   |   |   |   |   |--- RMS <= 0.46
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- RMS >  0.46
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- ZCR >  -1.21
|   |   |   |   |   |--- class: 0
|   |   |   |--- ZCR >  -1.16
|   |   |   |   |--- RMS <= -0.00
|   |   |   |   |   |--- RMS <= -0.09
|   |   |   |   |   |   |--- RMS <= -0.14
|   |   |   |   |   |   |   |--- RMS <= -0.19
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- RMS >  -0.19
|   |   |   |   |   |   |   |   |--- ZCR <= -0.99
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   

Model Lain


In [113]:
for model_name, model in models.items():
    print(f"{model_name} Model:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print("-" * 40)

rf Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
xg Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
cat Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
gb Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
lr Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
knn Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score: 0.7895
Confusion Matrix:
[[32  2]
 [ 6 15]]
----------------------------------------
dt Model:
Accuracy: 0.8545
Precision: 0.8824
Recall: 0.7143
F1 Score